In [1]:
import cv2
import os
import json
import numpy as np



file_path = '/data02/zhangrunxiang/reid1/fast-reid/orange_demo/video/0095_c2_t0175to0300/results.txt'

matched_results = []  # 用于存储符合条件的结果
data = []

with open(file_path, 'r') as file:
    for line in file:
        line_data = line.strip().split('\t')

        image_path = line_data[0]
        success = True if line_data[1] == 'True' else False
        similarity_distance = float(line_data[2])
        data.append([image_path, success, similarity_distance])

# 对所有数据进行检测
for item in data:
    image_path, success, similarity_distance = item

    parts = image_path.split('/')
    filename_part = parts[-1]
    filename_parts = filename_part.split('_')
    
    # 提取视角信息
    person_view = filename_parts[1]

    for part in filename_parts:
        if part.isdigit() and len(part) == 4:
            person_id_rerank = part
            frame_number = filename_parts[-1].split('.')[0]
            if similarity_distance < 0.4 and success:
                matched_results.append((person_id_rerank, frame_number, person_view))

# 打印所有符合条件的person_id_rerank、帧数和视角
for item in matched_results:
    print("Person ID Rerank: {}, Frame Number: {}, image View: {}".format(item[0], item[1], item[2]))
    

print("符合条件的结果数量为: ", len(matched_results))#符合条件结果有18

Person ID Rerank: 0095, Frame Number: t0190, image View: c2
Person ID Rerank: 0095, Frame Number: t0210, image View: c2
Person ID Rerank: 0095, Frame Number: t0215, image View: c2
Person ID Rerank: 0095, Frame Number: t0225, image View: c2
Person ID Rerank: 0095, Frame Number: t0230, image View: c2
Person ID Rerank: 0095, Frame Number: t0235, image View: c2
Person ID Rerank: 0095, Frame Number: t0240, image View: c2
Person ID Rerank: 0095, Frame Number: t0245, image View: c2
Person ID Rerank: 0095, Frame Number: t0250, image View: c2
Person ID Rerank: 0095, Frame Number: t0255, image View: c2
Person ID Rerank: 0095, Frame Number: t0260, image View: c2
Person ID Rerank: 0095, Frame Number: t0265, image View: c2
Person ID Rerank: 0095, Frame Number: t0270, image View: c2
Person ID Rerank: 0095, Frame Number: t0275, image View: c2
Person ID Rerank: 0095, Frame Number: t0280, image View: c2
Person ID Rerank: 0095, Frame Number: t0285, image View: c2
Person ID Rerank: 0095, Frame Number: t0

In [8]:

import json
# 指定JSON文件路径
json_file_path = '/data02/zhangrunxiang/reid1/fast-reid/scripts/r2o.json'

# 加载JSON文件中的数据
with open(json_file_path, 'r') as file:
    json_data = json.load(file)

person_id_rerank = item[0]
person_id = json_data.get(person_id_rerank)#这里就把rerank的id变成了原始id
print(person_id)

# 指定JSON文件目录路径
json_directory = '/data02/zhangrunxiang/reid1/datasets/Wildtrack_dataset_full/Wildtrack_dataset/annotations_positions/'

# 遍历matched_results，提取所有的帧数信息
frame_numbers_to_search = set(item[1] for item in matched_results)
desired_position = {}  # 用于存储行人ID、帧数和视角为5的信息

for frame_number in frame_numbers_to_search:
    # 构建要查找的JSON文件名
    json_file_name = '0000' + frame_number[1:] + '.json'
    json_file_path = json_directory + json_file_name

    # 加载JSON文件中的数据
    with open(json_file_path, 'r') as file:
        json_data = json.load(file)
        desired_info = None
        person_id = int(person_id)
        for person_info in json_data:
            if person_info.get("personID") == person_id:
                for view_info in person_info.get("views", []):
                    if view_info.get("viewNum") == 1:  # 视角为1对应c2
                        desired_position[(frame_number)] = view_info
                        break
print("desired_position: ", desired_position)



0136
desired_position:  {'t0295': {'viewNum': 1, 'xmax': 734, 'xmin': 347, 'ymax': 1203, 'ymin': 113}, 't0245': {'viewNum': 1, 'xmax': 991, 'xmin': 897, 'ymax': 368, 'ymin': 16}, 't0290': {'viewNum': 1, 'xmax': 820, 'xmin': 531, 'ymax': 973, 'ymin': 85}, 't0230': {'viewNum': 1, 'xmax': 975, 'xmin': 898, 'ymax': 302, 'ymin': 10}, 't0275': {'viewNum': 1, 'xmax': 937, 'xmin': 769, 'ymax': 634, 'ymin': 46}, 't0215': {'viewNum': 1, 'xmax': 992, 'xmin': 925, 'ymax': 260, 'ymin': 5}, 't0280': {'viewNum': 1, 'xmax': 926, 'xmin': 730, 'ymax': 724, 'ymin': 56}, 't0225': {'viewNum': 1, 'xmax': 963, 'xmin': 887, 'ymax': 292, 'ymin': 9}, 't0250': {'viewNum': 1, 'xmax': 987, 'xmin': 884, 'ymax': 406, 'ymin': 20}, 't0285': {'viewNum': 1, 'xmax': 877, 'xmin': 642, 'ymax': 833, 'ymin': 68}, 't0210': {'viewNum': 1, 'xmax': 955, 'xmin': 889, 'ymax': 250, 'ymin': 4}, 't0270': {'viewNum': 1, 'xmax': 956, 'xmin': 810, 'ymax': 562, 'ymin': 38}, 't0260': {'viewNum': 1, 'xmax': 973, 'xmin': 853, 'ymax': 471, '

In [9]:
import cv2
import os
import shutil

# 指定要处理的图像文件路径和保存路径
file_path = '/data02/zhangrunxiang/reid1/datasets/Wildtrack_dataset_full/Wildtrack_dataset/Image_subsets/C2/'
save_path = '/data02/zhangrunxiang/demo/baseline_image/baseline_gallery0095/'

# 创建保存路径文件夹
os.makedirs(save_path, exist_ok=True)

start_frame = 175
end_frame = 300
step = 5


# 复制文件到目标路径,视角下对应帧数图片复制到目标路径
for video_frame in range(start_frame, end_frame+1, step):
    source_file = os.path.join(file_path, f'{video_frame:08d}.png')  # 根据序号生成源文件路径
    destination_file = os.path.join(save_path, f'{video_frame:08d}.png')  # 根据序号生成目标文件路径
    shutil.copy(source_file, destination_file)


print("所有图片已成功复制到路径:", save_path)


# 遍历desired_position中所有帧数信息
for frame_number, info in desired_position.items():
    # 构建图像文件名
    image_filename = '0000' + frame_number[1:] + '.png'
    image_path = file_path + image_filename

    # 加载图像
    image = cv2.imread(image_path)

    # 获取坐标信息
    xmin = info['xmin']
    ymin = info['ymin']
    xmax = info['xmax']
    ymax = info['ymax']

    # 在图像上绘制矩形框
    cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)

    # 构建保存路径和文件名
    save_filename =  image_filename
    save_image_path = os.path.join(save_path, save_filename)

    # 保存带有框的图像
    cv2.imwrite(save_image_path, image, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])


print("带有框的图像已保存在路径:", save_path)


所有图片已成功复制到路径: /data02/zhangrunxiang/demo/baseline_image/gallery0095/
带有框的图像已保存在路径: /data02/zhangrunxiang/demo/baseline_image/gallery0095/


实现过程
1，相似距离大，判断没识别出来；相似距离小于阈值，判定为true，识别出来了
2，用rerank的id，找原始id,根据原始id和视角,在对应帧数找到相应视角的位置